In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import zcode.math as zmath
import zcode.plot as zplot

In [ ]:
# path = "/Users/lzkelley/programs/nanograv/ng15yr_astro_interp/ceffyl_ng15_multiorf_hdonly/"
path = "/Users/lzkelley/programs/nanograv/ceffyl/data/NG15/fs2A/"
freq = np.load(path + "freqs.npy")
dens = np.load(path + "density.npy").squeeze()
rho = np.load(path + "log10rhogrid.npy")
print(freq.shape, dens.shape, rho.shape)

In [ ]:
1/(freq*YR)

In [ ]:
CUT = 8

xx = freq[:CUT]
yy = (10.0 ** dens[:CUT])
yy = np.sqrt((xx[:, np.newaxis]**3)*yy)
yy = np.log10(yy)
xx = xx * YR

fig, ax = plot.figax(scale='lin') # , ylim=[-15, 2])
# plot._draw_plaw(ax, freq*YR, 1.0e-15, 1.0)
plt.violinplot(yy.T, positions=np.log10(xx))
plt.show()

In [ ]:
fig, ax = plot.figax()

dens_med = np.median(dens, axis=-1)
yy = 10.0 ** dens_med

ax.plot(freq, yy)
plt.show()

# temp = np.sqrt(yy * (freq) * (12*np.pi**2))
# print(temp)
# ax.plot(freq, temp)
# plt.show()

In [ ]:
fig, ax = plot.figax(scale='lin', ylim=[-20, 2])
# plot._draw_plaw(ax, freq*YR, 1.0e-15, 1.0)
plt.violinplot(dens.T, positions=np.log10(freq*YR))
plt.show()

In [ ]:
fname = "/Users/lzkelley/programs/nanograv/holodeck/output/hard04_2022-12-06_01/sam_lib.hdf5"
print(fname)
data = h5py.File(fname, 'r')
print("Keys:", data.keys())
for kk, vv in data.items():
    try:
        print("\t", kk, vv.shape)
    except AttributeError:
        continue
    
print("\nAttributes:")
for kk, vv in data.attrs.items():
    print("\t", kk, vv)
    
print("\nParameters:")
for kk, vv in data['parameters'].items():
    print("\t", kk, vv[:])

In [ ]:
fobs = data['fobs'][:]
xx = fobs * YR
fig, ax = plot.figax(xlabel='Frequency $[\\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

if True:
    # (nsamples, nfreqs, nreals)
    yy = data['gwb'][:]
    # ==> (nfreqs, nsamples, nreals)
    yy = np.moveaxis(yy, 1, 0)
    yy = np.median(yy, axis=-1)
    yy = yy.reshape(yy.shape[0], -1)
    ax.set(ylim=[1e-16, 3e-14])
    ax.plot(xx, yy, alpha=0.1, color='k', lw=1)

    for amp in [1e-15, 2e-15, 3e-15]:
        plot._draw_plaw(ax, xx, amp, 1.0, color='r', alpha=1.0, zorder=100)

if False:
    ff = zmath.argnearest(xx, 1.0)
    title = f"Characteristic Strain at ${xx[ff]:.2f} \, \\mathrm{{ yr}}^{{-1}}$"
    ax.set(xscale='linear', yscale='linear', xlim=[-17, -13], title=title)
    yy = data['gwb'][:, ff, :].flatten()
    yy = np.log10(yy[yy > 0.0])
    kale.dist1d(yy, ax=ax, density=True, carpet=True)

plt.show()

In [ ]:
data['gwb'].shape

In [ ]:
NUM = 60

fobs = data['fobs'][:]
xx = fobs * YR
fig, ax = plot.figax(xlabel='Frequency $[\\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)
ax.set(ylim=[1e-16, 3e-14])

# (nsamples, nfreqs, nreals)
yy = data['gwb'][:]

nsamp = yy.shape[0]
sel = np.random.choice(nsamp, NUM, replace=False)
yy = yy[sel]

# ==> (nfreqs, nsamples, nreals)
yy = np.moveaxis(yy, 1, 0)

med = np.median(yy, axis=-1)

for ii in range(NUM):
    cc, = ax.plot(xx, med[:, ii], alpha=0.5, lw=1) # , color='b')
    cc = cc.get_color()

    conf = np.percentile(yy[:, ii, :], [25, 75], axis=-1)
    ax.fill_between(xx, *conf, alpha=0.15, color=cc, lw=1)

    zz = yy[:, ii, :5]
    zz = zz.reshape(zz.shape[0], -1)
    ax.plot(xx, zz, alpha=0.25, color=cc, lw=1)

for amp in [1e-15, 2e-15, 3e-15]:
    hh = plot._draw_plaw(ax, xx, amp, 1.0, color='0.95', alpha=0.70, zorder=99, lw=2.0, ls='-')
    zplot.label_line(ax, hh[0], f"{zplot.scientific_notation(amp)}", x=0.85, dy=0.01, alpha=0.65, fontsize=10)
    plot._draw_plaw(ax, xx, amp, 1.0, color='r', alpha=0.75, zorder=100, lw=1.0)

plt.show()

In [ ]:
xx = data['fobs'][:] * YR
gwb = data['gwb'][:]
spars = data['sample_params'][:]
print(gwb.shape, spars.shape)

spars = spars.T
for ii, pname in enumerate(data.attrs['param_names']):
    fig, ax = plot.figax(xlabel="Frequency [1/yr]", ylabel="Characteristic Strain")
    plot._twin_hz(ax)

    pvals = data['parameters'][pname][:]
    print(ii, pname, pvals)
    sv = spars[ii].copy()
    sort_idx = np.argsort(sv)
    sv = sv[sort_idx]
    lo = 0

    num_pvals = len(pvals)
    colors = zplot.color_cycle(num_pvals, cmap='Spectral')

    for jj, pv in enumerate(pvals):
        # This boolean array is False until the first value ABOVE this bin
        hi = (sv > pv)
        # Find the index of the first value ABOVE of this bin
        hi = np.argmax(hi)
        # Make sure ANY of the values are above this bin
        hi = hi if (sv[hi] > pv) else None
        # Select the values since the last bin, up to the next bin
        cut = slice(lo, hi)
        # Select the GWB spectra in this bin, shape (samples, freqs, reals)
        yy = gwb[sort_idx][cut]
        # convert to shape (freqs, samples, reals)
        yy = np.moveaxis(yy, 1, 0)
        # convert to shape (freqs, samples*reals)
        yy = yy.reshape(yy.shape[0], -1)

        med = np.median(yy, axis=-1)
        conf = np.percentile(yy, [25, 75], axis=-1)
        ax.plot(xx, med, color=colors[jj], label=f"{pv:.1e}")
        ax.fill_between(xx, *conf, color=colors[jj], alpha=0.2)
        
        # set the lo-end of the next bin, to be the hi-end of this bin
        lo = hi

    ax.legend(title=pname)
    
    

In [ ]:
# kale.corner(data['params'][:].T)